<a href="https://www.kaggle.com/code/kamaljp/pytube-pandas-automate-n-analyse?scriptVersionId=91414130" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install pytube

     |████████████████████████████████| 56 kB 722 kB/s            


In [2]:
!pip install openpyxl

     |████████████████████████████████| 242 kB 884 kB/s            


In [3]:
!pip install xlrd

     |████████████████████████████████| 96 kB 984 kB/s            


In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from pytube import YouTube
from pytube import Channel
from pytube import Search
from pytube.innertube import InnerTube
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import openpyxl
import time
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# This notebook is for learning purpose only. Use the Pytube API with care. 

Scraping involves requests to Servers that are run by businesses. Businesses that can see our data and provide valuable information as incentive. Many businesses have opened their APIs for developers to consume the data generated. Pytube is one such API, which sends requests to Youtube servers. Be polite and request data by adding sufficient delay in your code and let the actual users enjoy the service.

Live and Let Live always... 

### <a id='mot'> Technical Motivation </a>

Democratisation of the Data Science happens with the Data. If we think where all this started, it will lead to Google Search. We spend countless hours searching for things online, in Google and Youtube. In such realtime searches, we leave the application once we get the result we look for. To get the final results, we do many searches by iterating through the keywords. 

Pytube automates the above activity, and provides the API to gather details about the results we might look at when doing the manual search. This notebook introduces one such automation, that I found very informative. Lets dive-in 

### What are the benefits???

By the end of this notebook, you will see how easy is to find answers to these questions and locate the most promising videos like a professional. You can view the [results here](#result)

1) How the Videos in each Channel is distributed?

2) Which channel's videos are mostly viewed?

3) Are there any one video that has ver high view count?

4) Can you get more videos about one channel?

5) Which video is having the longest play time? (The length provided in seconds)

There are more [opportunities](#opportunity) to extend the data received through the API

In [5]:
#It all begins with the search term.
my_search = 'Automated Warehouse India'

In [6]:
#Interesting method is completion_suggestions method, which provides a list of search terms
s = Search(my_search)
related_term = s.completion_suggestions
related_term.append(my_search)
print('1st Automation through the completion Auto suggestions')
print(len(related_term))

1st Automation through the completion Auto suggestions
15


### Couple of Helper Functions 

In [7]:
#Search class kickstarts the process, the methods results and get_next_results() are the workhorses
def get_youtube(search_term): #class takes the search term, gets the initial resultsa
    get_result = Search(search_term)
    ini_res = get_result.results
    while True: #collect fifty youtube videos
        get_result.get_next_results()
        ini_res = get_result.results
        if len(ini_res) > 50: #I am limiting my results to 50 links per each term
            break
    return ini_res

In [8]:
#Search lists of the pytube contains Youtube objects with the video_id at the end. We extract it
def videoSearch(search_list): #provide the search.results list as input
    vidlist = []
    for result in search_list:
        result = str(result)
        temp = result.replace("<pytube.__main__.YouTube object: videoId=", "")
        temp = temp.split(',')[0] #get the correct video id
        vidLink = ("https://www.youtube.com/watch?v=" + temp)
        vidlist.append(vidLink)
    return vidlist

In [9]:
#Get the links of each search terms that was listed by pytube
you_tube_links = []
for search_term in related_term:
    #get the raw links from get_youtube function
    temp_list = (get_youtube(search_term))
    #print('temp list length {}'.format(len(temp_list)))
    #temp list of each term is parsed into youtube links
    for link in videoSearch(temp_list):
        you_tube_links.append(link)
    #print('link list length {}'.format(len(you_tube_links)))

In [10]:
#Check and remove the links that are repeating. Simple method is to convert the list to set
print('Collected Links are {} in number '.format(len(you_tube_links)))
set_link = set(you_tube_links)
print('Collected Links duplicates removed are {} in number '.format(len(set_link)))
#convert back to list
you_tube_links = list(you_tube_links)

Collected Links are 845 in number 
Collected Links duplicates removed are 653 in number 


In [11]:
#Every youtube link has data like, author, title, views etc... We extract it as dataframe
def get_vid_data(link):
    vid_t = YouTube(link)
    keys = ','.join(vid_t.keywords)
    cha = Channel(vid_t.channel_url)
    data = {'Author':vid_t.author,'title':vid_t.title, 'Age_restriction':vid_t.age_restricted,'description':vid_t.description,
                'keywords':keys, 'length':vid_t.length,'publish_date':vid_t.publish_date,'views':vid_t.views,'about':cha.about_url}
    vid_df = pd.DataFrame(data,index=[0])
    return vid_df

In [12]:
#Youtube has channels. Each Channels have many videos under them. We can extract it. 
def dataframe_collector(linklist):
    vid_collector = pd.DataFrame()
    x = len(linklist)
    for link in linklist:
        c_data = get_vid_data(link)
        vid_collector = vid_collector.append(c_data)
        if vid_collector.shape[0]//10 == 0:
            print('Completing {} % of list'.format((vid_collector.shape[0]/x)*100))
    return vid_collector

### YouTube Streams and their benefits... (not covered in this NB)
Each youtube link is a treasure trove of Audio and Video data. The streams method provided by the pytube library, provides the list of available streams through the YouTube object created using the links. 

We wont be using the streams in this notebook. Video and Audio data can be useful for NLP, subject identification and Speec to text conversion tasks. Based on that text more analysis can be done 

In [13]:
#Function downloads the highest resolution audio from the link
def dload_aud(link):
    vid = YouTube(link)
    stream = vid.streams
    get_aud = stream.get_by_itag(140)
    get_aud.download()
    
#Function downloads the highest resolution video from the link    
def dload_vid(link):
    vid = YouTube(link)
    stream = vid.streams
    get_vid = stream.get_highest_resolution()
    get_vid.download()

In [14]:
#Get the data from each of the youtube links provided by the Search class. Here only less data is parsed 
final_dataframe = pd.DataFrame()
x = len(you_tube_links[:20])
#Can be unquoted and executed....
''' 
for collected_links in you_tube_links[:20]:
    
    #print(collected_links)
    
    final_dataframe = final_dataframe.append(get_vid_data(collected_links))
    check = round((final_dataframe.shape[0]/x)*100)
    
    if np.mod(check,10) == 0:
            print('Completing {} % of list'.format(check))
            time.sleep(15) #sleep for 15 seconds to avoid HTTP Error 429: Too Many Requests
'''

" \nfor collected_links in you_tube_links[:20]:\n    \n    #print(collected_links)\n    \n    final_dataframe = final_dataframe.append(get_vid_data(collected_links))\n    check = round((final_dataframe.shape[0]/x)*100)\n    \n    if np.mod(check,10) == 0:\n            print('Completing {} % of list'.format(check))\n            time.sleep(15) #sleep for 15 seconds to avoid HTTP Error 429: Too Many Requests\n"

In [15]:
#getting channel data...
def get_chnl_name(link):
    ch = YouTube(link)
    temp = ch.channel_url
    cha_temp = Channel(temp)
    time.sleep(3)
    return cha_temp.channel_name

In [16]:
#Attach the links to final dataframe, using only the 21 links recieved. Below code can be unquoted...
'''
final_dataframe['link'] = you_tube_links[:20]
final_dataframe['channel_name'] = final_dataframe.link.apply(lambda x: get_chnl_name(x))
final_dataframe.reset_index(inplace = True)
final_dataframe.to_csv('Final_dataframe.csv')
final_dataframe.head()
'''

"\nfinal_dataframe['link'] = you_tube_links[:20]\nfinal_dataframe['channel_name'] = final_dataframe.link.apply(lambda x: get_chnl_name(x))\nfinal_dataframe.reset_index(inplace = True)\nfinal_dataframe.to_csv('Final_dataframe.csv')\nfinal_dataframe.head()\n"

[Couple of Helper Functions](#mot) 

In [17]:
#The above program gets only partial data to avoid overwhelming the server. Lets use the existing data
Analy_dataframe = pd.read_csv('../input/pytube-data/Solar_panel_cleaning_robot.csv')

In [18]:
#getting channel data...
def get_chnl_vids(link):
    ch = Channel(link)
    num_url = len(ch.video_urls)
    time.sleep(5)
    return num_url

In [19]:
def get_chnl_name(link):
    ch = YouTube(link)
    temp = ch.channel_url
    cha_temp = Channel(temp)
    time.sleep(5)
    return cha_temp.channel_name

[Couple of Helper Functions](#mot) 

 ## <a id='result'> What insights can we get from the search results?</a>

In [20]:
#Visualisation are required, import plotly express and graph objects

import plotly.express as px
import plotly.graph_objects as go

In [21]:
#Ensure only the unique youtube video data enters our analysis.
print('There are total {} unique records'.format(len(Analy_dataframe.link.unique())))

There are total 483 unique records


In [22]:
#Using the drop duplicate method to get the dataframe
Analy_dataframe = Analy_dataframe.drop_duplicates(subset=['link'])

In [23]:
#get the channel description
analy_channel = Analy_dataframe.groupby(['channel_name','about'])['length'].count().reset_index()
analy_channel.sort_values('length',inplace=True,ascending=False)

In [24]:
#Which channels have more videos in this search?
cha = px.bar(data_frame=analy_channel[:20],y='channel_name',x='length',color='channel_name')
cha.update_layout(title='Channel and their distribution',height = 1000, width = 1000)
cha.show()

In [25]:
#Which channel videos are viewed the most?

#Lets consider only top 10 channels from the earlier question

top_chan = analy_channel.channel_name[:15]

vie = px.box(data_frame=Analy_dataframe[Analy_dataframe.channel_name.isin(top_chan)],
             x='views',color='channel_name')
vie.update_layout(title='Which channel videos are mostly viewed',height=800,width = 1000)
vie.show()

In [26]:
harsha = Analy_dataframe[Analy_dataframe.channel_name == 'Harsha solar panel cleaning']
#The Harsha Cleaning system seems to have a high view count
lin = px.bar(data_frame=harsha, x='title',y='views')
lin.update_layout(title='Only one video has miraculously got high view count')
lin.show()

In [27]:
#Lets check Eccopia
eco = Analy_dataframe[Analy_dataframe.channel_name == 'Ecoppia']
#The Harsha Cleaning system seems to have a high view count
lin = px.bar(data_frame=eco, x='title',y='views')
lin.update_layout(title='How about Eccopia?')
lin.show()

In [28]:
#Get video lengths to locate if there is any informational vids.
inf_len = Analy_dataframe.groupby('channel_name')['length'].mean().reset_index()
inf_len.sort_values('length',inplace=True,ascending=False)
#Visualising 50 channels with long videos
inf_bar = px.bar(data_frame=inf_len[:20], x='channel_name',y='length')
inf_bar.update_layout(title='Which Channels have longest videos?')
inf_bar.show()

In [29]:
top_info = inf_len.channel_name
#Is it possible to find informative videos from the data? 
info = px.box(data_frame=Analy_dataframe,x='length',color='channel_name')
info.update_layout(title='Which channels have longest videos?')
info.show()

In [30]:
Analy_dataframe['vid_id'] = Analy_dataframe.link.apply(lambda x: x.split('=')[1])
Analy_dataframe.sort_values('length',inplace=True,ascending=False)
vid_bar = px.bar(data_frame=Analy_dataframe[:100], x='vid_id',y='length')
vid_bar.update_layout(title='Which are the longest videos?')
vid_bar.show()

### <a id='opportunity'> Opportunities are endless... </a>

The description provided, the access to other videos of each channels and their description can be a starting point of 

- QnA bot,

- Semantic analysis 

- Word Cloud visualisation

- Further improving search using the keywords.

All geared towards taking the search automation much easier for the bots that we will build.